In [1]:
import pandas as pd
import json

## **Pre process match results**

In [2]:
with open(r"E:\Data Science\Power BI\Sqade selection\t20_json_files\t20_wc_match_results.json") as f:
    data = json.load(f)
df_match = pd.DataFrame(data[0]['matchSummary'])
df_match.head()

,team1,team2,winner,margin,ground,matchDate,scorecard
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022",T20I # 1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,"Oct 16, 2022",T20I # 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022",T20I # 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,"Oct 17, 2022",T20I # 1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,"Oct 18, 2022",T20I # 1830


In [3]:
## For PK and FK have to use use 'scorecard' 
df_match.rename({'scorecard' : 'match_id'} , axis = 1 , inplace = True)

In [4]:
df_match.to_csv(r'E:\Data Science\Power BI\Sqade selection\Pre Peocesed Data\match_summary.csv' , index = False)

## **Pre process Batting info**

In [5]:
with open(r"E:\Data Science\Power BI\Sqade selection\t20_json_files\t20_wc_batting_summary.json") as f:
    data = json.load(f)

    all_records = []

    for rec in data:
        all_records.extend(rec['battingSummary'])

df_batting = pd.DataFrame(all_records)
df_batting.head()

,match,teamInnings,battingPos,batsmanName,dismissal,runs,balls,4s,6s,SR
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,c Pramod Madushan b Chameera,3,6,0,0,50.00
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,c Shanaka b Pramod Madushan,9,9,1,0,100.00
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,c â€ Mendis b Karunaratne,20,12,1,2,166.66
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,c DM de Silva b Pramod Madushan,26,24,2,0,108.33
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),c Gunathilaka b PWH de Silva,20,24,0,0,83.33


In [6]:
##Convert dusmissal column into Our or Not out
df_batting['Out/not_out'] = df_batting.dismissal.apply(lambda x: "OUT" if len(x) > 0 else "NOT OUT")
df_batting.drop(columns=['dismissal'] , inplace = True)


In [7]:
df_batting.head(10)

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,Out/not_out
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,3,6,0,0,50.00,OUT
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,9,9,1,0,100.00,OUT
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,OUT
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,OUT
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),20,24,0,0,83.33,OUT
5,Namibia Vs Sri Lanka,Namibia,6,Jan Frylinck,44,28,4,0,157.14,OUT
6,Namibia Vs Sri Lanka,Namibia,7,David Wiese,0,1,0,0,0.00,OUT
7,Namibia Vs Sri Lanka,Namibia,8,JJ Smit,31,16,2,2,193.75,NOT OUT
8,Namibia Vs Sri Lanka,Sri Lanka,1,Pathum Nissanka,9,10,1,0,90.00,OUT
9,Namibia Vs Sri Lanka,Sri Lanka,2,Kusal Mendisâ€,6,6,0,0,100.00,OUT


In [8]:
df_batting['batsmanName'] = df_batting['batsmanName'].apply(lambda x: x.replace('â€' , ' '))

df_batting['batsmanName'] = df_batting['batsmanName'].apply(lambda x: x.replace('â€' , ' '))
df_batting.head()

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,Out/not_out
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,3,6,0,0,50.00,OUT
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,9,9,1,0,100.00,OUT
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,OUT
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,OUT
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),20,24,0,0,83.33,OUT


**Create a match Id for connecting two table**

In [9]:
## Create a match id according to the team1 and team2
match_id_dict = {}

for index , row in df_match.iterrows():
    key1 = row['team1'] + ' Vs ' + row['team2']
    key2 = row['team2'] + 'Vs ' + row['team1']

    match_id_dict[key1] = row["match_id"]
    match_id_dict[key2] = row["match_id"]


In [10]:
##Assosiate the match id with the batsman table
df_batting["match_id"] = df_batting["match"].map(match_id_dict) 
df_batting.head(10)

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,Out/not_out,match_id
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,3,6,0,0,50.00,OUT,T20I # 1823
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,9,9,1,0,100.00,OUT,T20I # 1823
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,OUT,T20I # 1823
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,OUT,T20I # 1823
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),20,24,0,0,83.33,OUT,T20I # 1823
5,Namibia Vs Sri Lanka,Namibia,6,Jan Frylinck,44,28,4,0,157.14,OUT,T20I # 1823
6,Namibia Vs Sri Lanka,Namibia,7,David Wiese,0,1,0,0,0.00,OUT,T20I # 1823
7,Namibia Vs Sri Lanka,Namibia,8,JJ Smit,31,16,2,2,193.75,NOT OUT,T20I # 1823
8,Namibia Vs Sri Lanka,Sri Lanka,1,Pathum Nissanka,9,10,1,0,90.00,OUT,T20I # 1823
9,Namibia Vs Sri Lanka,Sri Lanka,2,Kusal Mendis,6,6,0,0,100.00,OUT,T20I # 1823


In [11]:
df_batting.to_csv(r'E:\Data Science\Power BI\Sqade selection\Pre Peocesed Data\batting_summary.csv' , index = False)

## **Pre process bolling summary**

In [13]:
with open(r"E:\Data Science\Power BI\Sqade selection\t20_json_files\t20_wc_bowling_summary.json") as f:
    data = json.load(f)
    all_records = []
    for roc in data:
        all_records.extend(roc['bowlingSummary'])
all_records[:2]

[{'match': 'Namibia Vs Sri Lanka',
  'bowlingTeam': 'Sri Lanka',
  'bowlerName': 'Maheesh Theekshana',
  'overs': '4',
  'maiden': '0',
  'runs': '23',
  'wickets': '1',
  'economy': '5.75',
  '0s': '7',
  '4s': '0',
  '6s': '0',
  'wides': '2',
  'noBalls': '0'},
 {'match': 'Namibia Vs Sri Lanka',
  'bowlingTeam': 'Sri Lanka',
  'bowlerName': 'Dushmantha Chameera',
  'overs': '4',
  'maiden': '0',
  'runs': '39',
  'wickets': '1',
  'economy': '9.75',
  '0s': '6',
  '4s': '3',
  '6s': '1',
  'wides': '2',
  'noBalls': '0'}]

In [14]:
df_bowling = pd.DataFrame(all_records)
df_bowling.head()

,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls
0,Namibia Vs Sri Lanka,Sri Lanka,Maheesh Theekshana,4,0,23,1,5.75,7,0,0,2,0
1,Namibia Vs Sri Lanka,Sri Lanka,Dushmantha Chameera,4,0,39,1,9.75,6,3,1,2,0
2,Namibia Vs Sri Lanka,Sri Lanka,Pramod Madushan,4,0,37,2,9.25,6,3,1,0,0
3,Namibia Vs Sri Lanka,Sri Lanka,Chamika Karunaratne,4,0,36,1,9.00,7,3,1,1,0
4,Namibia Vs Sri Lanka,Sri Lanka,Wanindu Hasaranga de Silva,4,0,27,1,6.75,8,1,1,0,0


In [15]:
df_bowling['match_id'] = df_bowling['match'].map(match_id_dict)
df_bowling.head()

,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,match_id
0,Namibia Vs Sri Lanka,Sri Lanka,Maheesh Theekshana,4,0,23,1,5.75,7,0,0,2,0,T20I # 1823
1,Namibia Vs Sri Lanka,Sri Lanka,Dushmantha Chameera,4,0,39,1,9.75,6,3,1,2,0,T20I # 1823
2,Namibia Vs Sri Lanka,Sri Lanka,Pramod Madushan,4,0,37,2,9.25,6,3,1,0,0,T20I # 1823
3,Namibia Vs Sri Lanka,Sri Lanka,Chamika Karunaratne,4,0,36,1,9.00,7,3,1,1,0,T20I # 1823
4,Namibia Vs Sri Lanka,Sri Lanka,Wanindu Hasaranga de Silva,4,0,27,1,6.75,8,1,1,0,0,T20I # 1823


In [16]:
df_bowling.to_csv(r'E:\Data Science\Power BI\Sqade selection\Pre Peocesed Data\bowling_summary.csv' , index = False)

## **Pre process player info**

In [18]:
with open(r"E:\Data Science\Power BI\Sqade selection\t20_json_files\t20_wc_player_info.json") as f:
    data = json.load(f)

df_players = pd.DataFrame(data)
df_players.drop(columns='description' , inplace=True)

df_players.head(10)

,name,team,battingStyle,bowlingStyle,playingRole
0,Michael van Lingen,Namibia,Left hand Bat,Left arm Medium,Bowling Allrounder
1,Divan la Cock,Namibia,Right hand Bat,Legbreak,Opening Batter
2,Jan Nicol Loftie-Eaton,Namibia,Left hand Bat,"Right arm Medium, Legbreak",Batter
3,Stephan Baard,Namibia,Right hand Bat,Right arm Medium fast,Batter
4,Gerhard Erasmus(c),Namibia,Right hand Bat,Right arm Offbreak,Allrounder
5,Jan Frylinck,Namibia,Left hand Bat,Left arm Fast medium,Allrounder
6,David Wiese,Namibia,Right hand Bat,Right arm Medium fast,Allrounder
7,JJ Smit,Namibia,Right hand Bat,Left arm Medium fast,Bowling Allrounder
8,Pathum Nissanka,Sri Lanka,Right hand Bat,,Top order Batter
9,Kusal Mendisâ€,Sri Lanka,Right hand Bat,Legbreak,Wicketkeeper Batter


In [19]:
df_batting['batsmanName'] = df_batting['batsmanName'].apply(lambda x: x.replace('â€' , ' '))

In [20]:
df_players.to_csv(r'E:\Data Science\Power BI\Sqade selection\Pre Peocesed Data\players_summary.csv' , index = False)